In [ ]:
#| default_exp data.database.sqlalchemy

In [ ]:
from geopy.timezone import from_timezone_name
#| hide
from nbdev.showdoc import *
from pyasn1_modules.rfc3279 import id_fieldType

In [ ]:
#| hide
import nbdev;

nbdev.nbdev_export()

In [ ]:
#| hide
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import os
import pandas as pd

In [ ]:
from sqlalchemy import MetaData, create_engine, asc, desc, and_, or_, not_, case, extract, cast, text
from sqlalchemy.types import DateTime, Date, Time
from sqlalchemy.schema import *
from sqlalchemy.sql import func as F, Selectable, select
from sqlalchemy.dialects import registry
from sqlalchemy.engine.row import Row
from sqlalchemy.orm import Session

In [ ]:
#| export
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials


In [ ]:
#| hide
from google.cloud import bigquery
from dotenv import load_dotenv


In [ ]:

load_dotenv("../.env")

In [ ]:
os.environ

In [ ]:
# os.getenv('GC_QUOTE_API_CREDENTIALS')
# os.getenv('HTTP_PROXY')
# os.getenv('HTTPS_PROXY')

In [ ]:
# config = dotenv_values("../.env")
# key = config.get('GC_QUOTE_API_CREDENTIALS')
key = os.getenv('GC_QUOTE_API_CREDENTIALS')

In [ ]:
credential = service_account.Credentials.from_service_account_file("../" + key)


In [ ]:

#os.environ['HTTP_PROXY'] = "http://127.0.0.1:20171"
#os.environ['HTTPS_PROXY'] = "http://127.0.0.1:20171"
#os.environ['http_proxy'] = "http://127.0.0.1:20171"
#os.environ['https_proxy'] = "http://127.0.0.1:20171"



In [ ]:
registry.register('bigquery', 'sqlalchemy_bigquery', 'BigQueryDialect')
engine = create_engine('bigquery://quote-api-365206',
                       credentials_path='../' + key, )


In [ ]:
metadata = MetaData()


def get_table(project_name: str, dataset_name: str, table_name: str) -> Table:
    table = Table(f'{project_name}.{dataset_name}.{table_name}', metadata, autoload_with=engine)
    return table

In [ ]:
# table = get_table('quote-api-365206', 'report', 'ride_dispatch_v')

In [ ]:
# query = select(table).limit(20).where(table.c.ride_stat == 'Accepted').order_by(asc(table.c.ride_id))
# df = pd.read_sql(query, engine)
# df


In [ ]:
# table = get_table('elife-data-warehouse-prod', 'ods', 'ride_ride')

In [ ]:
# table

In [ ]:
# query = select(table).where(table.c.airport_code3 == 'JFK').limit(20)

In [ ]:
# df = pd.read_sql(query, engine)
# df


In [ ]:
# find all tables in the database

In [ ]:
ride_trip_t = get_table('elife-data-warehouse-prod', 'ods', 'ride_trip').alias()
ride_dispatch_t = get_table('elife-data-warehouse-prod', 'ods', 'ride_dispatch').alias()
ride_enum_t = get_table('elife-data-warehouse-prod', 'ods', 'ride_enum').alias()
ride_ride_t = get_table('elife-data-warehouse-prod', 'ods', 'ride_ride').alias()
ride_partner_tran_t = get_table('elife-data-warehouse-prod', 'ods', 'ride_partner_tran').alias()
ride_partner_t = get_table('elife-data-warehouse-prod', 'ods', 'ride_partner').alias()
dim_place_t = get_table('elife-data-warehouse-prod', 'dim', 'dim_place').alias()
ride_vehicle_class_t = get_table('elife-data-warehouse-prod', 'ods', 'ride_vehicle_class').alias()


In [ ]:
auction_ride_t = get_table('elife-data-warehouse-prod', 'ods', 'ride_auction_ride').alias()
auction_fleet_t = get_table('elife-data-warehouse-prod', 'ods', 'ride_auction_fleet').alias()
ride_fleet_t = get_table('elife-data-warehouse-prod', 'ods', 'ride_fleet').alias()
dim_service_area_t = get_table('elife-data-warehouse-prod', 'dim', 'dim_service_area').alias()
ride_cur_xrate_history_t = get_table('elife-data-warehouse-prod', 'ods', 'ride_cur_exch_rate_history').alias()



In [ ]:
ride_cur_xrate_t = select(ride_ride_t.c.id.label('ride_id'),
                          ride_dispatch_t.c.inserted_at,
                          ride_cur_xrate_history_t.c.from_cur,
                          ride_cur_xrate_history_t.c.to_cur,
                          ride_cur_xrate_history_t.c.rate,
                          ride_cur_xrate_history_t.c.effective_at,
                          ride_cur_xrate_history_t.c.expire_at)
ride_cur_xrate_t = ride_cur_xrate_t.select_from(ride_ride_t
                                                .join(ride_dispatch_t, ride_ride_t.c.id == ride_dispatch_t.c.ride_id)
                                                .join(ride_cur_xrate_history_t,
                                                      ride_dispatch_t.c.inserted_at.between(
                                                          ride_cur_xrate_history_t.c.effective_at,
                                                          ride_cur_xrate_history_t.c.expire_at),
                                                      )
                                                ).where(
    and_(
        # ride_dispatch_t.c.inserted_at.between(ride_cur_xrate_history_t.c.effective_at, ride_cur_xrate_history_t.c.expire_at),
        ride_cur_xrate_history_t.c.from_cur == 'USD',
        ride_cur_xrate_history_t.c.to_cur == ride_dispatch_t.c.currency
    )
)
# ride_cur_xrate_history_t.c.effective_at <= ride_dispatch_t.c.inserted_at,
# ride_dispatch_t.c.inserted_at == ride_cur_xrate_history_t.c.inserted_at,

# and_(
#     ride_cur_xrate_history_t.c.effective_at <= ride_dispatch_t.c.inserted_at,
#     ride_dispatch_t.c.inserted_at < ride_cur_xrate_history_t.c.expire_at
# ),
ride_cur_xrate_t = ride_cur_xrate_t.alias()



In [ ]:

# print(ride_cur_xrate_t)
# ride_cur_xrate_t = ride_cur_xrate_t.limit(100)
# df = pd.read_sql(ride_cur_xrate_t, engine)
# df

In [ ]:
# ride_cur_xrate_t = select(ride_ride_t.c.id.label('ride_id'),
#                           ride_dispatch_t.c.inserted_at,
#                           ride_cur_xrate_history_t.c.from_cur,
#                           ride_cur_xrate_history_t.c.to_cur,
#                           ride_cur_xrate_history_t.c.rate,
#                           ride_cur_xrate_history_t.c.effective_at,
#                           ride_cur_xrate_history_t.c.expire_at).where(
#     and_(
#         ride_dispatch_t.c.inserted_at.between(ride_cur_xrate_history_t.c.effective_at, ride_cur_xrate_history_t.c.expire_at),
#         ride_cur_xrate_history_t.c.from_cur == 'USD',
#         ride_cur_xrate_history_t.c.to_cur == ride_dispatch_t.c.currency
#     )
# )
# ride_cur_xrate_t = ride_cur_xrate_t.select_from(ride_ride_t
#                                                 .join(ride_dispatch_t, ride_ride_t.c.id == ride_dispatch_t.c.ride_id)
#                                                 .join(ride_cur_xrate_history_t, 
#                                                       ride_dispatch_t.c.inserted_at.between(ride_cur_xrate_history_t.c.effective_at, 
#                                                                                             ride_cur_xrate_history_t.c.expire_at),
#                                                       isouter=True)
#                                                 )
#                                                       # ride_cur_xrate_history_t.c.effective_at <= ride_dispatch_t.c.inserted_at,
#                                                       # ride_dispatch_t.c.inserted_at == ride_cur_xrate_history_t.c.inserted_at,
#                                                       
#                                                       # and_(
#                                                       #     ride_cur_xrate_history_t.c.effective_at <= ride_dispatch_t.c.inserted_at,
#                                                       #     ride_dispatch_t.c.inserted_at < ride_cur_xrate_history_t.c.expire_at
#                                                       # ),
# ride_cur_xrate_t = ride_cur_xrate_t.alias()



In [ ]:
trip_type_id_t = select(ride_dispatch_t.c.ride_id, ride_trip_t.c.trip_type.label('trip_type_id'))
trip_type_id_t = trip_type_id_t.select_from(ride_dispatch_t
                                            .join(ride_trip_t, ride_dispatch_t.c.ride_id == ride_trip_t.c.ride_id))
trip_type_id_t = trip_type_id_t.alias()


In [ ]:
# trip_type_id_t = trip_type_id_t.limit(10)
# df = pd.read_sql(trip_type_id_t, engine)
# df

In [ ]:
trip_type_t = select(trip_type_id_t.c.ride_id, trip_type_id_t.c.trip_type_id, ride_enum_t.c.name.label('trip_type'))
trip_type_t = trip_type_t.select_from(trip_type_id_t
                                      .join(ride_enum_t, trip_type_id_t.c.trip_type_id == ride_enum_t.c.id,
                                            isouter=True))
trip_type_t = trip_type_t.alias()


In [ ]:
ride_status_t = select(ride_ride_t.c.id.label('ride_id'), ride_ride_t.c.stat.label('ride_status_id'),
                       ride_enum_t.c.name.label('ride_status'))
ride_status_t = ride_status_t.select_from(ride_ride_t
                                          .join(ride_enum_t, ride_ride_t.c.stat == ride_enum_t.c.id, isouter=True))
ride_status_t = ride_status_t.alias()
# ride_status_t = ride_status_t.limit(10)
# df = pd.read_sql(ride_status_t, engine)
# df

In [ ]:

service_area_t = select(ride_ride_t.c.id.label('ride_id'), ride_ride_t.c.service_area_id,
                        dim_service_area_t.c.name.label('service_area'))
service_area_t = service_area_t.select_from(ride_ride_t
                                            .join(dim_service_area_t,
                                                  ride_ride_t.c.service_area_id == dim_service_area_t.c.id))
service_area_t = service_area_t.alias()
# service_area_t = service_area_t.limit(10)
# df = pd.read_sql(service_area_t,engine)
# df.dtypes
# df
print(service_area_t)


## Traffic peak time exclusion
source https://www.quora.com/What-is-the-trickiest-time-of-the-day-to-drive-in-Orlando

In [ ]:
#ride_datetime_tz_t = select(ride_datetime_t.c.ride_id,
#                         # F.timestamp(ride_datetime_t.c.from_datetime).label('from_datetime_utc'),
#                         ride_datetime_t.c.from_datetime,
#                         ride_datetime_t.c.from_datetime_str,
#                         cast(ride_datetime_t.c.from_date_str, Date).label('from_date'),
#                         cast(ride_datetime_t.c.from_time_str, Time).label('from_time'),
#                         # ride_datetime_t.c.from_date_str,
#                         # ride_datetime_t.c.from_time_str,
#                         ride_datetime_t.c.utc,
#                         ride_datetime_t.c.tz,
#                         ride_datetime_t.c.day_of_week,
#                         F.datetime(F.timestamp_seconds(ride_datetime_t.c.utc)).label('from_datetime_utc'),
#                         #F.format_timestamp('%Y-%m-%d %H:%M:%S', F.timestamp_seconds(ride_datetime_t.c.utc), ride_datetime_t.c.tz).label('datetime_with_new_timezone')
#                         # text("TIMESTAMP(ride_datetime_t.c.from_datetime) AT TIME ZONE 'UTC' AT TIME ZONE ride_datetime_t.c.tz").label('datetime_with_new_timezone') 
#                         #F.datetime(F.timestamp_seconds(ride_datetime_t.c.utc), F.timezone(ride_datetime_t.c.tz)).label('from_datetime_local'),
#                         # cast(ride_datetime_t.c.from_datetime_str, DateTime).label('from_datetime_tz'),
#                         )
#ride_datetime_tz_t = ride_datetime_tz_t.select_from(ride_datetime_t) #.join(ride_ride_t, ride_ride_t.c.id == ride_datetime_t.c.ride_id))
## print(ride_non_peak_t.compile(compile_kwargs={"literal_binds": True}))
#ride_datetime_tz_t = ride_datetime_tz_t.alias()
## ride_datetime_tz_t = ride_datetime_tz_t.limit(100)
## df = pd.read_sql(ride_datetime_tz_t,engine)
## df.dtypes
## df

In [ ]:
ride_datetime_t = select(ride_ride_t.c.id.label('ride_id'),
                         F.substring(ride_ride_t.c.from_time_str, 1, 10).label('from_date_str'),
                         F.concat(F.substring(ride_ride_t.c.from_time_str, 12, 16), ':00').label('from_time_fix_str'),
                         F.concat(F.substring(ride_ride_t.c.from_time_str, 1, 10),
                                  ' ',
                                  F.substring(ride_ride_t.c.from_time_str, 12, 16),
                                  ':00'
                                  ).label('from_datetime_fix_str'),
                         # extract(
                         #     'DAYOFWEEK',
                         #     cast(F.concat(F.substring(ride_ride_t.c.from_time_str, 1, 10),
                         #                   ' ',
                         #                   F.substring(ride_ride_t.c.from_time_str, 12, 16),
                         #                   # ':00'
                         #                   ),
                         #          DateTime)
                         # ).label('day_of_week_local'),
                         extract(
                             'DAYOFWEEK',
                             cast(F.timestamp_seconds(ride_ride_t.c.from_utc),
                                  DateTime)
                         ).label('day_of_week_utc'),
                         F.datetime(F.timestamp_seconds(ride_ride_t.c.from_utc)).label('from_datetime_utc'),
                         ride_ride_t.c.from_timezone_str,
                         # F.current_date(ride_ride_t.c.from_timezone_str).label('current_date'),
                         #cast(ride_ride_t.c.from_time_str,DateTime).label('from_datetime'),
                         #cast(F.concat(F.substring(ride_ride_t.c.from_time_str, 1, 10), 'T', F.substring(ride_ride_t.c.from_time_str, 12, 16)),DateTime).label('from_datetime'),
                         #cast(ride_ride_t.c.from_time_str, DateTime).label('from_time_dt'),
                         # F.timezone(ride_ride_t.c.from_timezone_str, cast(ride_ride_t.c.from_time_str, DateTime).label('from_time_dt')),
                         )
ride_datetime_t = ride_datetime_t.select_from(ride_ride_t)
ride_datetime_t = ride_datetime_t.alias()
# ride_datetime_t = ride_datetime_t.limit(100)
# df = pd.read_sql(ride_datetime_t, engine)

In [ ]:
#df

In [ ]:
from workalendar.usa import Florida
import numpy as np

cal_florida = Florida()
exclude_dates_str = [str(d[0]) for d in cal_florida.holidays(2024)]
exclude_dates_str

In [ ]:
#ride_non_peak_t = select(ride_datetime_t).where(
#            # not_(ride_datetime_tz_t.c.day_of_week.in_([1, 7])),
#            # not_(ride_datetime_tz_t.c.from_datetime_str.in_(exclude_dates)),
#            and_(
#                # not_(ride_datetime_t.c.day_of_week.in_([1, 7])),
#                not_(ride_datetime_t.c.from_datetime_str.in_(exclude_dates_str)),
#                not_(
#                    and_(
#                        # text(f"TIME({ride_datetime_tz_t.c.from_time}) > TIME({peak_time_str[0][0]}) AND TIME({ride_datetime_tz_t.c.from_time}) < TIME({peak_time_str[0][1]})"),
#                        # text(f"TIME({ride_datetime_tz_t.c.from_time}) > TIME({peak_time_str[1][0]}) AND TIME({ride_datetime_tz_t.c.from_time}) < TIME({peak_time_str[1][1]})"),
#                        cast(ride_datetime_t.c.from_time, Time) > cast(peak_time_str[0][0],Time),
#                        cast(ride_datetime_t.c.from_time, Time) < cast(peak_time_str[0][1],Time),
#                    )
#                ),
#                not_(
#                    and_(
#                        cast(ride_datetime_t.c.from_time, Time) > cast(peak_time_str[1][0],Time),
#                        cast(ride_datetime_t.c.from_time, Time) < cast(peak_time_str[1][1],Time),
#                    )
#                )
#            )
#        )               
#ride_non_peak_t = ride_non_peak_t.select_from(ride_datetime_t)
## ride_non_peak_t = ride_non_peak_t.alias()
#ride_non_peak_t = ride_non_peak_t.limit(1000)
#df = pd.read_sql(ride_non_peak_t,engine)
#df.head()

In [ ]:
# df

In [ ]:
exclude_dates = [d[0] for d in cal_florida.holidays(2024)]
# exclude_dates
# res = df['from_datetime_utc'].apply(lambda x: x in exclude_dates)

In [ ]:
# np.any(res.apply(lambda x: x in exclude_dates))
# res[0]=True
# res
# np.any(res)


In [ ]:
dispatch_status_t = select(ride_dispatch_t.c.id.label('ride_id'), ride_dispatch_t.c.stat.label('dispatch_status_id'),
                           ride_enum_t.c.name.label('dispatch_status'))
dispatch_status_t = dispatch_status_t.select_from(ride_dispatch_t
                                                  .join(ride_enum_t, ride_dispatch_t.c.stat == ride_enum_t.c.id,
                                                        isouter=True))
dispatch_status_t = dispatch_status_t.alias()
#dispatch_status_t = dispatch_status_t.limit(10)
#df = pd.read_sql(dispatch_status_t, engine)
#df

In [ ]:

expr = case(
    (ride_dispatch_t.c.to_fleet_id == auction_fleet_t.c.fleet_id, 'auction'),
    else_='dispatch',
)


In [ ]:
auction_type_t = select(
    auction_ride_t.c.ride_id.label('ride_id'),
    auction_ride_t.c.auction_id.label('auction_id'),
    auction_fleet_t.c.fleet_id.label('auction_fleet_id'),
    ride_dispatch_t.c.to_fleet_id.label('dispatch_fleet_id'),
    ride_fleet_t.c.name.label('fleet'),
    case(
        (ride_dispatch_t.c.to_fleet_id == auction_fleet_t.c.fleet_id, 'auction'),
        else_='dispatch',
    ).label('dispatch_type')
)
auction_type_t = auction_type_t.select_from(auction_ride_t
                                            .join(auction_fleet_t,
                                                  auction_ride_t.c.auction_id == auction_fleet_t.c.auction_id,
                                                  isouter=True)
                                            .join(ride_dispatch_t,
                                                  auction_ride_t.c.ride_id == ride_dispatch_t.c.ride_id, isouter=True)
                                            .join(ride_fleet_t, ride_dispatch_t.c.to_fleet_id == ride_fleet_t.c.id,
                                                  isouter=True))
auction_type_t = auction_type_t.alias()
# auction_type_t = auction_type_t.limit(100)
# df = pd.read_sql(auction_type_t, engine)
# df

In [ ]:
dispatch_type_t = (select(
    ride_dispatch_t.c.id.label('ride_id'),
    ride_dispatch_t.c.to_fleet_id.label('dispatch_fleet_id'),
    auction_ride_t.c.auction_id.label('auction_id'),
    auction_fleet_t.c.fleet_id.label('auction_fleet_id'),
    case(
        (ride_dispatch_t.c.to_fleet_id == auction_fleet_t.c.fleet_id, 'auction'),
        else_='dispatch',
    ).label('dispatch_type'),
    ride_fleet_t.c.name.label('fleet')
))
# .where(
#     ride_dispatch_t.c.to_fleet_id == auction_fleet_t.c.fleet_id
# ))
dispatch_type_t = dispatch_type_t.select_from(ride_dispatch_t
                                              .join(auction_ride_t,
                                                    ride_dispatch_t.c.ride_id == auction_ride_t.c.ride_id, isouter=True)
                                              .join(auction_fleet_t,
                                                    auction_ride_t.c.auction_id == auction_fleet_t.c.auction_id,
                                                    isouter=True)
                                              .join(ride_fleet_t, ride_dispatch_t.c.to_fleet_id == ride_fleet_t.c.id,
                                                    isouter=True))
dispatch_type_t = dispatch_type_t.alias()
# dispatch_type_t = dispatch_type_t.limit(100)
# df = pd.read_sql(dispatch_type_t, engine)
# df

In [ ]:
partner_id_t = select(ride_ride_t.c.id.label('ride_id'), ride_partner_tran_t.c.partner_id.label('partner_id'))
partner_id_t = partner_id_t.select_from(ride_ride_t
                                        .join(ride_partner_tran_t,
                                              ride_ride_t.c.partner_tran_id == ride_partner_tran_t.c.id, isouter=True))
partner_id_t = partner_id_t.alias()
#partner_id_t = partner_id_t.limit(10)
#df = pd.read_sql(partner_id_t, engine)
#df

In [ ]:
partner_t = select(partner_id_t.c.ride_id, partner_id_t.c.partner_id, ride_partner_t.c.name.label('partner'))
partner_t = partner_t.select_from(partner_id_t
                                  .join(ride_partner_t, partner_id_t.c.partner_id == ride_partner_t.c.id, isouter=True))
partner_t = partner_t.alias()
# partner_t = partner_t.limit(10)
# df = pd.read_sql(partner_t,engine)
# df
# print(partner_t)

In [ ]:
from_place_t = select(ride_ride_t.c.id.label('ride_id'),
                      ride_ride_t.c.from_place_id.label('start_place_id'),
                      dim_place_t.c.name.label('start_place'),
                      dim_place_t.c.lng.label('lng'),
                      dim_place_t.c.lat.label('ltt'),
                      )
from_place_t = from_place_t.select_from(ride_ride_t
                                        .join(dim_place_t, ride_ride_t.c.from_place_id == dim_place_t.c.id,
                                              isouter=True))
from_place_t = from_place_t.alias()
#from_place_t = from_place_t.limit(10)
#df = pd.read_sql(from_place_t, engine)
#df

In [ ]:
to_place_t = select(ride_ride_t.c.id.label('ride_id'),
                    ride_ride_t.c.to_place_id.label('end_place_id'),
                    dim_place_t.c.name.label('end_place'),
                    dim_place_t.c.lng.label('lng'),
                    dim_place_t.c.lat.label('ltt'))
to_place_t = to_place_t.select_from(ride_ride_t
                                    .join(dim_place_t, ride_ride_t.c.to_place_id == dim_place_t.c.id, isouter=True))
to_place_t = to_place_t.alias()
#to_place_t = to_place_t.limit(10)
#df = pd.read_sql(to_place_t, engine)
#df

In [ ]:
vehicle_class_t = select(ride_ride_t.c.id.label('ride_id'),
                         ride_ride_t.c.vehicle_class_id.label('vehicle_class_id'),
                         ride_vehicle_class_t.c.name.label('vehicle_class'))
vehicle_class_t = vehicle_class_t.select_from(ride_ride_t
                                              .join(ride_vehicle_class_t,
                                                    ride_ride_t.c.vehicle_class_id == ride_vehicle_class_t.c.id,
                                                    isouter=True))
vehicle_class_t = vehicle_class_t.alias()
#vehicle_class_t = vehicle_class_t.limit(10)
#df = pd.read_sql(vehicle_class_t, engine)
#df

In [ ]:
from workalendar.usa import Florida

In [ ]:
cal_orlando = Florida()
data_from_date = '2024-01-01'


In [ ]:
exclude_dates = [d[0] for d in cal_orlando.holidays(2024)]
exclude_dates

In [ ]:
price_training_t = select(ride_ride_t.c.id.label('ride_id'),
                          ride_ride_t.c.trip_count,
                          ride_ride_t.c.from_utc,
                          ride_ride_t.c.from_time_str,
                          ride_ride_t.c.from_timezone_str,
                          ride_ride_t.c.to_time_str,
                          ride_ride_t.c.to_timezone_str,
                          ride_ride_t.c.passenger_count,
                          ride_ride_t.c.luggage_count,
                          ride_ride_t.c.children_count,
                          ride_ride_t.c.infant_count,
                          ride_ride_t.c.distance,
                          ride_ride_t.c.duration,
                          ride_dispatch_t.c.id.label('dispatch_id'),
                          ride_dispatch_t.c.trip_no,
                          ride_dispatch_t.c.amount.label('dispatch_amount'),
                          ride_dispatch_t.c.currency.label('dispatch_currency'),
                          ride_dispatch_t.c.inserted_at,
                          ride_cur_xrate_history_t.c.from_cur,
                          ride_cur_xrate_history_t.c.to_cur,
                          ride_cur_xrate_history_t.c.rate,
                          ride_cur_xrate_history_t.c.effective_at,
                          ride_cur_xrate_history_t.c.expire_at,
                          # ride_cur_xrate_t.c.inserted_at,
                          # ride_cur_xrate_t.c.from_cur,
                          # ride_cur_xrate_t.c.to_cur,
                          # ride_cur_xrate_t.c.rate,
                          # ride_cur_xrate_t.c.effective_at,
                          # ride_cur_xrate_t.c.expire_at,
                          ride_datetime_t.c.from_date_str,
                          ride_datetime_t.c.from_time_fix_str,
                          ride_datetime_t.c.from_datetime_fix_str,
                          # text("EXTRACT(DAYOFWEEK FROM TIMESTAMP_SECONDS(ride_ride_t.from_utc))").label('day_of_week'),
                          trip_type_t.c.trip_type_id,
                          trip_type_t.c.trip_type,
                          ride_status_t.c.ride_status_id,
                          ride_status_t.c.ride_status,
                          dispatch_status_t.c.dispatch_status_id,
                          dispatch_status_t.c.dispatch_status,
                          dispatch_type_t.c.dispatch_type,
                          ride_fleet_t.c.name.label('fleet'),
                          partner_t.c.partner_id,
                          partner_t.c.partner,
                          service_area_t.c.service_area_id,
                          service_area_t.c.service_area,
                          from_place_t.c.start_place_id,
                          from_place_t.c.start_place,
                          from_place_t.c.lng.label('start_lng'),
                          from_place_t.c.ltt.label('start_ltt'),
                          to_place_t.c.end_place_id,
                          to_place_t.c.end_place,
                          to_place_t.c.lng.label('end_lng'),
                          to_place_t.c.ltt.label('end_ltt'),
                          vehicle_class_t.c.vehicle_class_id,
                          vehicle_class_t.c.vehicle_class
                          )

In [ ]:
price_training_t = price_training_t.select_from(ride_ride_t
                                                .join(ride_dispatch_t, ride_ride_t.c.id == ride_dispatch_t.c.ride_id)
                                                # .join(ride_cur_xrate_history_t,
                                                #       ride_dispatch_t.c.inserted_at.between(
                                                #           ride_cur_xrate_history_t.c.effective_at,
                                                #           ride_cur_xrate_history_t.c.expire_at),
                                                #       )
                                                .join(ride_datetime_t, ride_ride_t.c.id == ride_datetime_t.c.ride_id)
                                                # .join(ride_cur_xrate_t, ride_ride_t.c.id == ride_cur_xrate_t.c.ride_id, isouter=True)
                                                .join(trip_type_t, ride_ride_t.c.id == trip_type_t.c.ride_id,
                                                      isouter=True)
                                                .join(ride_status_t, ride_ride_t.c.id == ride_status_t.c.ride_id,
                                                      isouter=True)
                                                .join(dispatch_status_t,
                                                      ride_ride_t.c.id == dispatch_status_t.c.ride_id, isouter=True)
                                                .join(dispatch_type_t, ride_ride_t.c.id == dispatch_type_t.c.ride_id,
                                                      isouter=True)
                                                .join(ride_fleet_t, ride_dispatch_t.c.to_fleet_id == ride_fleet_t.c.id,
                                                      isouter=True)
                                                .join(partner_t, ride_ride_t.c.id == partner_t.c.ride_id, isouter=True)
                                                .join(service_area_t, ride_ride_t.c.id == service_area_t.c.ride_id)
                                                .join(from_place_t, ride_ride_t.c.id == from_place_t.c.ride_id,
                                                      isouter=True)
                                                .join(to_place_t, ride_ride_t.c.id == to_place_t.c.ride_id,
                                                      isouter=True)
                                                .join(vehicle_class_t, ride_ride_t.c.id == vehicle_class_t.c.ride_id,
                                                      isouter=True)
                                                ).where(
    and_(
        ride_ride_t.c.from_time_str > data_from_date,
        # ride_dispatch_t.c.currency == 'USD',
        # vehicle_class_t.c.vehicle_class== 'MPV-5',
        or_(dispatch_type_t.c.dispatch_type == 'auction',
            dispatch_type_t.c.dispatch_type == 'dispatch'),
        # ride_dispatch_t.c.inserted_at.between(ride_cur_xrate_history_t.c.effective_at, ride_cur_xrate_history_t.c.expire_at),
        and_(
            ride_dispatch_t.c.inserted_at.between(ride_cur_xrate_history_t.c.effective_at, ride_cur_xrate_history_t.c.expire_at),
            ride_cur_xrate_history_t.c.from_cur == 'USD',
            ride_cur_xrate_history_t.c.to_cur == ride_dispatch_t.c.currency,
        ),
        # ride_cur_xrate_history_t.c.from_cur == 'USD',
        # ride_cur_xrate_history_t.c.to_cur == ride_dispatch_t.c.currency
        # F.lower(from_place_t.c.start_place).like('%orlando%'),
        # F.lower(to_place_t.c.end_place).like('%orlando%'),

        # not_(ride_datetime_t.c.day_of_week.in_([1, 7])),
        # not_(ride_ride_t.c.from_time_str.in_(exclude_dates)),
    )
)
#price_training_t = price_training_t.where(ride_ride_t.c.from_time_str > '2024-01-01')
price_training_t = price_training_t.limit(10)


In [ ]:
# print(price_training_t)
print(price_training_t.compile())

In [ ]:

print(price_training_t.compile().params)


In [ ]:
df = pd.read_sql(price_training_t, engine)
df

In [ ]:
sqlite_eng = create_engine('sqlite:///../data/price_training_from_gbq_raw.db', echo=False)

In [ ]:
df.to_sql('price_training_orlando_mpv5', sqlite_eng, if_exists='replace')

In [ ]:
from pricing.data.utils import validate_datetime_in_iso_format, validate_timezone_in_iana, get_timezone_abbreviation, \
    fix_timezone

In [ ]:
df_invalid_datetime = df[df.apply(lambda x: not validate_datetime_in_iso_format(x['from_datetime_fix_str']), axis=1)]
df_invalid_datetime

In [ ]:
df_invalid_timezone = df[df.apply(lambda x: not validate_timezone_in_iana(x['from_timezone_str']), axis=1)]
df_invalid_timezone

In [ ]:
#orlando_airport = pd.read_csv('../../data/orlando_all_output.csv')
#orlando_airport.head()
#orlando_airport.dtypes
#orlando_airport.to_sql('orlando_airport', sqlite_eng, if_exists='append')

In [ ]:
df['from_timezone_fix_str'] = df.apply(lambda x: fix_timezone(x['from_timezone_str']), axis=1)
df

In [ ]:

df_valid_datetime = df[df.apply(lambda x: validate_datetime_in_iso_format(x['from_datetime_fix_str']), axis=1)]
df_valid_timezone = df_valid_datetime[
    df_valid_datetime.apply(lambda x: validate_timezone_in_iana(x['from_timezone_fix_str']), axis=1)]
df_valid_timezone


In [ ]:

df_valid_timezone.loc[df_invalid_timezone.index, ['from_timezone_str', 'from_timezone_fix_str']]


In [ ]:
from datetime import datetime
import pytz

df_utc_offset = (df_valid_timezone.apply(lambda x: datetime.now(pytz.timezone(x['from_timezone_fix_str'])), axis=1)
                 .apply(lambda x: x.strftime('%z')))
df_utc_offset.name = 'utc_offset'
df_utc_offset


In [ ]:
df_dt_str = df_valid_timezone['from_datetime_fix_str']
df_valid_timezone['from_datetime_local'] = df_valid_timezone.apply(
    lambda x: (pd.to_datetime(x['from_datetime_fix_str']).to_datetime64()), axis=1)
df_valid_timezone

In [ ]:
df_valid_timezone.dtypes

In [ ]:

# df_valid_timezone['from_datetime_local_tz'] = df_valid_timezone.apply(lambda x: pytz.timezone(x.loc[:,'from_timezone_str']).localize(x.loc[:,'from_datetime_local']), axis=1)
df_valid_timezone['from_timezone'] = df_valid_timezone.apply(lambda x: pytz.timezone(x['from_timezone_fix_str']),
                                                             axis=1)
df_valid_timezone


In [ ]:
df_valid_timezone['from_datetime_tz'] = df_valid_timezone.apply(
    lambda x: x['from_timezone'].localize(x['from_datetime_local']), axis=1)
df_valid_timezone
# .apply(lambda x: x.localize(x.loc[:,'from_timezone_str']), axis=1))
# df_valid_timezone


# filtering out peak traffic time

In [ ]:
peak_time_str = [('07:00:00', '09:00:00'), ('16:00:00', '19:00:00')]
night_time_str = [('22:00:00', '6:00:00')]  # Shanghai, US usually no overtime extra fees New York 8pm ~ 6am
ind = []
td = []
for pt in peak_time_str:
    ind.append(pd.DatetimeIndex(pt))
ind
for i in ind:
    i[1] - i[0]

In [ ]:
peak_time = []
for pt in peak_time_str:
    peak_time.append(pd.date_range(pt[0], pt[1], freq='h'))
for pt in peak_time:
    print(pt, pt.time)

In [ ]:
peak_time_str

In [ ]:

df_peak_traffic_time = df_valid_timezone[
    df_valid_timezone['from_datetime_tz'].apply(lambda x: x.strftime('%H:%M:%S')).between(*peak_time_str[0])
    | df_valid_timezone['from_datetime_tz'].apply(lambda x: x.strftime('%H:%M:%S')).between(*peak_time_str[1])
    ]
df_peak_traffic_time

In [ ]:

df_out_of_peak_traffic_time = df_valid_timezone[
    ~ (df_valid_timezone['from_datetime_tz'].apply(lambda x: x.strftime('%H:%M:%S')).between(*peak_time_str[0])
       | df_valid_timezone['from_datetime_tz'].apply(lambda x: x.strftime('%H:%M:%S')).between(*peak_time_str[1]))
]
df_out_of_peak_traffic_time


In [ ]:

# df_utc_offset = (df_valid_timezone.apply(lambda x: datetime.now(pytz.timezone(x.loc['timezone'])))
#                             .apply(lambda x: x.strftime('%z')))
# df_utc_offset.name = 'utc_offset'
# df_utc_offset
# 

# filter out round trip (time reservation) with feature distance = 1 (m)

In [ ]:
df_no_round_trip = df_valid_timezone[df_valid_timezone['distance'] > 1]

In [ ]:
df_training = df_no_round_trip.loc[:,
              ['ride_id', 'trip_type', 'trip_no', 'trip_count', 'ride_status', 'partner', 'fleet',
               'start_place', 'end_place',
               'passenger_count', 'luggage_count',
               'dispatch_amount', 'dispatch_currency',
               'distance', 'duration', 'vehicle_class',
               'from_datetime_tz']]

df_training['cent_price_per_km'] = df_training['dispatch_amount'] / df_training['distance'] * 100.0
df_training

In [ ]:
def average_cent_per_km(x):
    x['average_cent_per_km'] = x['cent_price_per_km'].mean()
    return x

In [ ]:
def fleet_trip_no(x):
    x['fleet_trip_count'] = len(x)
    return x

In [ ]:
df_fleet_statistics = df_training.loc[:, ['ride_id', 'fleet']]
df_fleet_statistics

In [ ]:

df_fleet_trip_no = df_fleet_statistics.groupby('fleet').aggregate([len])
df_fleet_trip_no.sort_values(by=('ride_id', 'len'), ascending=False, inplace=True)
df_fleet_trip_no


In [ ]:
df_big_fleets = df_fleet_trip_no[df_fleet_trip_no[('ride_id', 'len')] > 100]
df_big_fleets

In [ ]:
df_big_fleet_data = []
for f in df_big_fleets.index:
    print(f)
    df_big_fleet_data.append(df_training[df_training['fleet'] == f])

In [ ]:
df_big_fleet_data[0]

In [ ]:
df_big_fleet_data[1]

In [ ]:
df_big_fleet_data[2]

In [ ]:

# df_training['average_cent_per_km'] = df_training.groupby('fleet').apply(average_cent_per_km)
df_analysis = df_training.groupby('fleet').apply(average_cent_per_km)
df_analysis


In [ ]:

df_training

In [ ]:
#| hide
import nbdev;

nbdev.nbdev_export()